In [ ]:
import os


In [ ]:
%pwd


'/Users/tumelo/Desktop/ML Projects/youtube/dswithbappy/mlops_dl_cv_chest_ct/research'

In [ ]:
os.chdir("../")


In [ ]:
%pwd


'/Users/tumelo/Desktop/ML Projects/youtube/dswithbappy/mlops_dl_cv_chest_ct'

## Entity


In [ ]:
from dataclasses import dataclass
from pathlib import Path


In [ ]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


## Configuration


In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


## Component


In [ ]:
import os
import zipfile
import gdown

from cnnClassifier.utils.common import get_size

from cnnClassifier.logger import LoggerManager


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.logger = LoggerManager(self.__class__.__name__).get_logger()
        self.config = config

    def download_file(self) -> str:
        """
        Fetch the data from the url
        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            self.logger.info(
                f"Downloading data from {dataset_url} into filw {zip_download_dir}"
            )

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)
            self.logger.info(
                f"Downloaded data from {dataset_url} into filw {zip_download_dir}"
            )

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


## Pipeline


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-06-30 00:35:04] utils/common - read_yaml:33 - INFO - yaml file: config/config.yaml loaded successfully
[2025-06-30 00:35:04] utils/common - read_yaml:33 - INFO - yaml file: params.yaml loaded successfully
[2025-06-30 00:35:04] utils/common - create_directories:53 - INFO - created directory at: artifacts
[2025-06-30 00:35:04] utils/common - create_directories:53 - INFO - created directory at: artifacts/data_ingestion
[2025-06-30 00:35:04] DataIngestion - download_file:15 - INFO - Downloading data from https://drive.google.com/file/d/1ziMc35v-b1mAjuvNH4ZBCz5gNmztp76l/view?usp=share_link into filw artifacts/data_ingestion/data.zip
Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1ziMc35v-b1mAjuvNH4ZBCz5gNmztp76l
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1ziMc35v-b1mAjuvNH4ZBCz5gNmztp76l&confirm=t&uuid=45abbd0a-ddef-4296-8467-8f44981ad933
To: /Users/tumelo/Desktop/ML Projects/youtube/dswithbappy/mlops_dl_cv_chest_ct/artifacts/